In [1]:
import pandas as pd

def to_list(list_of_strings):
    return list(map(float, list_of_strings.strip('][').split(', ')))

def treat_pcts(pcts):
    for col in ['pct_multilevel_2', 'pct_multilevel_3', 'pct_multilevel_4']:
        pcts[col] = pcts[col].apply(to_list)
    return pcts

def get_pcts(pcts, file):
    pcts_file = (pcts[pcts['file name'] == file]).iloc[0]
    return(pcts_file['pct_200'], pcts_file['pct_500'], pcts_file['pct_800'], pcts_file['pct_1600'],
            pcts_file['pct_multilevel_2'], pcts_file['pct_multilevel_3'], pcts_file['pct_multilevel_4'])

In [2]:
import csv
import mido
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import math
%matplotlib inline

import sys
sys.path.insert(0, '../../scripts')

import stft_zoom, display, detect_musical_regions
from util import *
import mappings
import pickle
import PIL
import IPython.display
from classes import SingleResSpectrogram, MultiResSpectrogram
from aug_density_map import *
from mappings import *

/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/Library/Python/3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [3]:
import librosa
import numpy as np
import mido

import sys
sys.path.insert(0, '../../scripts')

from util import *
from aug_density_map import *
from mappings import *

import string

def msg2dict(msg):
    result = dict()
    if 'note_on' in msg:
        on_ = True
    elif 'note_off' in msg:
        on_ = False
    else:
        on_ = None
    result['time'] = int(msg[msg.rfind('time'):].split(' ')[0].split('=')[1].translate(
        str.maketrans({a: None for a in string.punctuation})))

    if on_ is not None:
        for k in ['note', 'velocity']:
            result[k] = int(msg[msg.rfind(k):].split(' ')[0].split('=')[1].translate(
                str.maketrans({a: None for a in string.punctuation})))
    return [result, on_]

def switch_note(last_state, note, velocity, on_=True):
    # piano has 88 notes, corresponding to note id 21 to 108, any note out of this range will be ignored
    result = [0] * 88 if last_state is None else last_state.copy()
    if 21 <= note <= 108:
        result[note-21] = velocity if on_ else 0
    return result

def get_new_state(new_msg, last_state):
    new_msg, on_ = msg2dict(str(new_msg))
    new_state = switch_note(last_state, note=new_msg['note'], velocity=new_msg['velocity'], on_=on_) if on_ is not None else last_state
    return [new_state, new_msg['time']]
def track2seq(track):
    # piano has 88 notes, corresponding to note id 21 to 108, any note out of the id range will be ignored
    result = []
    last_state, last_time = get_new_state(str(track[0]), [0]*88)
    for i in range(1, len(track)):
        new_state, new_time = get_new_state(track[i], last_state)
        if new_time > 0:
            result += [last_state]*new_time
        last_state, last_time = new_state, new_time
    return result

def mid2arry(mid, timestamp, min_msg_pct=0.1):
    tracks_len = [len(tr) for tr in mid.tracks]
    min_n_msg = max(tracks_len) * min_msg_pct
    # convert each track to nested list
    all_arys = []
    for i in range(len(mid.tracks)):
        if len(mid.tracks[i]) > min_n_msg:
            ary_i = track2seq(mid.tracks[i])
            all_arys.append(ary_i)
    # make all nested list the same length
    max_len = max([len(ary) for ary in all_arys])
    for i in range(len(all_arys)):
        if len(all_arys[i]) < max_len:
            all_arys[i] += [[0] * 88] * (max_len - len(all_arys[i]))
    all_arys = np.array(all_arys)
    all_arys = all_arys.max(axis=0)
    proll_proto = np.where(all_arys>0, 1, 0).T
    proll = np.zeros([127, proll_proto.shape[1]])
    proll[21:109,:] = proll_proto
    
    start_time = int(mido.second2tick(timestamp, tempo=500000, ticks_per_beat=mid.ticks_per_beat))
    stop_time  = int(mido.second2tick(timestamp+30, tempo=500000, ticks_per_beat=mid.ticks_per_beat))
    
    return proll[:,start_time:stop_time]

In [4]:
def get_refine_map(midi_file, kernel, timestamp, num_time_frames, n_fft=512, hop_size=512):
    midi = mido.MidiFile(midi_file)
    proll = mid2arry(midi, timestamp)
    proll_den = calc_map_aug2(sound_event_to_tfp(proll, num_time_frames, n_fft=n_fft), kernel, type='avg', n_fft=n_fft, hop_size=hop_size)
    return np.array(proll_den > 0, dtype=int)

def get_pct_to_refine(midi_file, kernel, timestamp, n_fft=512, hop_size=512):
    num_time_frames = int(np.ceil(44100 * 30 / hop_size))
    refine_map = get_refine_map(midi_file, kernel, timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
    return 100*(np.sum(refine_map)/refine_map.size)

# fazer: overlap dos detectados com kernel grande e todos kernel pequeno: N
#        overlap dos detectados com kernel pequeno e detectados kernel grande: n
#        o numero que queremos é n/N

# no caso real: N continua o mesmo, overlap dos detectado com kernel pequeno e refinados kernel grande = n

def in_range_list(range, range_list):
    for r in range_list:
        if range[0][0] >= r[0][0] and range[0][1] <= r[0][1]:
            if range[1][0] >= r[1][0] and range[1][1] <= r[1][1]:
                return True
    return False

def all_overlapping_ranges(range_list_1, range_list_2):
    # returns ranges in list_1 that are contained inside the regions defined by range_list_2
    overlapping_ranges = []
    for range in range_list_1:
        if in_range_list(range, range_list_2):
            overlapping_ranges.append(range)
    return overlapping_ranges

def get_pct_to_refine_multilevel(midi_file, kernel_list, timestamp, n_fft=512, hop_size=512, y=None):
# 1. pegar kernel primeiro nivel, ver no ground truth as regioes relevantes, guardar os ranges.
# 2. pegar kernel segundo, ver no ground truth as regioes relevantes, guardar os ranges.
# 3. ver overlap entre passo 2 e 1 = n
# 4. pegar kernel segundo, pegar ranges de todas as subregioes definidas por ele
# 5. ver overlap entre passo 4 e 1 = N
# 6. calcular pct, guardar regiões do passo 3 como as novas subregioes refinadas.
    spec = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_size))
    x_axis, y_axis = stft_zoom.get_axes_values(44100, 0, [0.0, 30.0], spec.shape) 
    num_time_frames = int(np.ceil(44100 * 30 / hop_size))
    first_level = get_refine_map(midi_file, kernel_list[0], timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
    pct_list = [100*(np.sum(first_level)/first_level.size)]
    indices_first_level = np.where(first_level.flatten() > 0)[0]
    range_list_first = detect_musical_regions.musical_regions_to_ranges(indices_first_level, first_level.shape, x_axis, y_axis, kernel_list[0], n_fft=n_fft, hop_size=hop_size)
    for kernel in kernel_list[1:]:
        second_level = get_refine_map(midi_file, kernel, timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
        indices_second_level = np.where(second_level.flatten() > 0)[0]
        all_subregions_second_level = np.where(second_level.flatten() > -1)[0]
        all_ranges_second_level = detect_musical_regions.musical_regions_to_ranges(all_subregions_second_level, second_level.shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)
        range_list_second = detect_musical_regions.musical_regions_to_ranges(indices_second_level, second_level.shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)
        n = len(all_overlapping_ranges(range_list_second, range_list_first))
        N = len(all_overlapping_ranges(all_ranges_second_level, range_list_first))
        pct_list.append((n/N) * 100)
        range_list_first = range_list_second
    return pct_list

def all_overlapping_ranges_idx(range_list_1, range_list_2):
    # returns ranges in list_1 that are contained inside the regions defined by range_list_2
    overlapping_ranges = []
    i = 0
    for range in range_list_1:
        if in_range_list(range, range_list_2):
            overlapping_ranges.append(i)
        i += 1
    return overlapping_ranges

def get_overlapping_regions(midi_file, kernel_list, timestamp, n_fft=512, hop_size=512, y=None):
# 1. pegar kernel primeiro nivel, ver no ground truth as regioes relevantes, guardar os ranges.
# 2. pegar kernel segundo, ver no ground truth as regioes relevantes, guardar os ranges.
# 3. ver overlap entre passo 2 e 1 = n
# 4. pegar kernel segundo, pegar ranges de todas as subregioes definidas por ele
# 5. ver overlap entre passo 4 e 1 = N
# 6. calcular pct, guardar regiões do passo 3 como as novas subregioes refinadas.
    spec = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_size))
    x_axis, y_axis = stft_zoom.get_axes_values(44100, 0, [0.0, 30.0], spec.shape) 
    num_time_frames = int(np.ceil(44100 * 30 / hop_size))
    first_level = get_refine_map(midi_file, kernel_list[0], timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
    pct_list = [100*(np.sum(first_level)/first_level.size)]
    indices_first_level = np.where(first_level.flatten() > 0)[0]
    range_list_first = detect_musical_regions.musical_regions_to_ranges(indices_first_level, first_level.shape, x_axis, y_axis, kernel_list[0], n_fft=n_fft, hop_size=hop_size)
    for kernel in kernel_list[1:]:
        second_level = get_refine_map(midi_file, kernel, timestamp, num_time_frames, n_fft=n_fft, hop_size=hop_size)
        indices_second_level = np.where(second_level.flatten() > 0)[0]
        all_subregions_second_level = np.where(second_level.flatten() > -1)[0]
        all_ranges_second_level = detect_musical_regions.musical_regions_to_ranges(all_subregions_second_level, second_level.shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)
        range_list_second = detect_musical_regions.musical_regions_to_ranges(indices_second_level, second_level.shape, x_axis, y_axis, kernel, n_fft=n_fft, hop_size=hop_size)
        n = all_overlapping_ranges_idx(range_list_second, range_list_first)
        n_idx = [np.unravel_index(x, second_level.shape) for x in n]
        N = all_overlapping_ranges_idx(all_ranges_second_level, range_list_first)
        N_idx = [np.unravel_index(x, second_level.shape) for x in N]
        range_list_first = range_list_second
    return n_idx, N_idx

In [5]:
pcts = pd.read_csv('./results/pcts.csv', sep=';')
pcts = treat_pcts(pcts)

In [6]:
deu_ruim = pcts[pcts['pct_1600'] > 45.0]['file name']

In [7]:
file_list = deu_ruim.to_numpy()

In [8]:
len(file_list)

279

In [ ]:
# result_file = './results/pcts_new_over_smart.csv'
# with open(result_file, 'w') as csvfile:
#     writer = csv.writer(csvfile, delimiter=';')
#     writer.writerow(['file name', 'pct_200', 'pct_500', 'pct_800', 'pct_1600', 'pct_multilevel_4'])
#     print('CSV OVERWRITTEN')

In [10]:
j = 157

for file in file_list[157:]:
    y, sr = librosa.load(file + '.wav', sr=44100)
    timestamp = int((len(y) / sr) // 2) # pega o meio da gravação (só analisaremos 30 segundos do meio da música)
    y = y[sr*timestamp:sr*(timestamp+30)]
    print('\n\n', j, file)
    print('Processing file ', j, 'of ', len(file_list))
    j += 1

    n_fft = 512
    hop_size = n_fft
    midi = file + '.midi'

    try:
        pct_200 = get_pct_to_refine(midi, [200,200], timestamp, n_fft=n_fft, hop_size=hop_size)
        pct_500 = get_pct_to_refine(midi, [500,500], timestamp, n_fft=n_fft, hop_size=hop_size)
        pct_800 = get_pct_to_refine(midi, [800,800], timestamp, n_fft=n_fft, hop_size=hop_size)
        pct_1600 = get_pct_to_refine(midi, [1600,1600], timestamp, n_fft=n_fft, hop_size=hop_size)
        
        print('pcts:', pct_200, pct_500, pct_800, pct_1600)

#         pct_multilevel_2 = get_pct_to_refine_multilevel(midi, [[1600,1600], [800,800]], timestamp, n_fft=n_fft, hop_size=n_fft, y=y)
#         pct_multilevel_3 = get_pct_to_refine_multilevel(midi, [[1600,1600], [800,800], [400,400]], timestamp, n_fft=n_fft, hop_size=n_fft, y=y)
        pct_multilevel_4 = get_pct_to_refine_multilevel(midi, [[1600,1600], [800,800], [400,400], [200,200]], timestamp, n_fft=n_fft, hop_size=n_fft, y=y)
        print('multipcts:', pct_multilevel_4)
            
    except:
        print("erro nos pct")
        continue


    result_file = './results/pcts_new_over_smart.csv'
    with open(result_file, 'a') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow([file, pct_200, pct_500, pct_800, pct_1600, pct_multilevel_4])



 157 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2011/MIDI-Unprocessed_04_R2_2011_MID--AUDIO_R2-D2_03_Track03_wav
Processing file  157 of  279
pcts: 8.026315789473685 18.92156862745098 29.27927927927928 50.0
multipcts: [50.0, 63.23529411764706, 60.588235294117645, 54.93827160493827]


 158 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2011/MIDI-Unprocessed_03_R3_2011_MID--AUDIO_R3-D1_07_Track07_wav
Processing file  158 of  279
pcts: 9.154135338345865 17.058823529411764 21.62162162162162 34.25925925925926
multipcts: [34.25925925925926, 65.38461538461539, 66.66666666666666, 62.83333333333333]


 159 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2011/MIDI-Unprocessed_09_R1_2011_MID--AUDIO_R1-D3_15_Track15_wav
Processing file  159 of  279
pcts: 11.82330827067669 22.745098039215687 31.756756756756754 47.22222222222222
multipcts: [47.22222222222222, 63.793103448275865, 55.494505494505496, 61.3738738738

pcts: 8.646616541353383 19.11764705882353 26.8018018018018 43.51851851851852
multipcts: [43.51851851851852, 62.264150943396224, 51.26582278481012, 60.32448377581121]


 182 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2011/MIDI-Unprocessed_11_R1_2011_MID--AUDIO_R1-D4_07_Track07_wav
Processing file  182 of  279
pcts: 7.913533834586466 14.019607843137255 20.495495495495494 39.81481481481482
multipcts: [39.81481481481482, 52.88461538461539, 55.64516129032258, 68.24104234527687]


 183 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2011/MIDI-Unprocessed_03_R1_2011_MID--AUDIO_R1-D1_18_Track18_wav
Processing file  183 of  279
pcts: 10.996240601503759 22.058823529411764 30.630630630630627 55.55555555555556
multipcts: [55.55555555555556, 57.14285714285714, 60.0, 59.43396226415094]


 184 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2011/MIDI-Unprocessed_15_R1_2011_MID--AUDIO_R1-D6_10_Track10_wav
Processing 

multipcts: [45.370370370370374, 55.00000000000001, 67.3469387755102, 59.424920127795524]


 206 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2013/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_17_R1_2013_wav--4
Processing file  206 of  279
pcts: 8.139097744360903 16.372549019607842 22.07207207207207 42.592592592592595
multipcts: [42.592592592592595, 48.148148148148145, 65.04065040650406, 59.75232198142415]


 207 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2013/ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_02_R2_2013_wav--3
Processing file  207 of  279
pcts: 7.87593984962406 18.03921568627451 23.873873873873876 48.148148148148145
multipcts: [48.148148148148145, 55.08474576271186, 61.36363636363637, 51.973684210526315]


 208 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2013/ORIG-MIDI_02_7_8_13_Group__MID--AUDIO_14_R2_2013_wav--4
Processing file  208 of  279
pcts: 6.353383458646617 12.058823529411764 16.2162162162

pcts: 7.011278195488722 14.313725490196077 17.792792792792792 37.03703703703704
multipcts: [37.03703703703704, 48.888888888888886, 61.458333333333336, 57.56302521008403]


 231 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2014/MIDI-UNPROCESSED_01-03_R1_2014_MID--AUDIO_03_R1_2014_wav--5
Processing file  231 of  279
pcts: 7.969924812030076 15.686274509803921 22.07207207207207 41.66666666666667
multipcts: [41.66666666666667, 50.0, 56.557377049180324, 62.664473684210535]


 232 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2014/MIDI-UNPROCESSED_11-13_R1_2014_MID--AUDIO_13_R1_2014_wav--6
Processing file  232 of  279
pcts: 8.026315789473685 19.80392156862745 27.7027027027027 49.074074074074076
multipcts: [49.074074074074076, 60.86956521739131, 58.536585365853654, 61.386138613861384]


 233 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2014/MIDI-UNPROCESSED_04-08-12_R3_2014_MID--AUDIO_04_R3_2014_wav--1
Pro

multipcts: [46.2962962962963, 68.33333333333333, 54.666666666666664, 55.19713261648745]


 255 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_10_R1_2015_wav--2
Processing file  255 of  279
pcts: 6.447368421052632 15.88235294117647 25.225225225225223 43.51851851851852
multipcts: [43.51851851851852, 66.66666666666666, 54.19847328244275, 61.60714285714286]


 256 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2015/MIDI-Unprocessed_R1_D1-9-12_mid--AUDIO-from_mp3_09_R1_2015_wav--3
Processing file  256 of  279
pcts: 9.699248120300751 20.68627450980392 27.47747747747748 49.074074074074076
multipcts: [49.074074074074076, 54.385964912280706, 59.25925925925925, 52.16836734693877]


 257 /Volumes/HD-NICO/vaio-backup/Documents/ime/compmus/mestrado/maestro-v2.0.0/2015/MIDI-Unprocessed_R1_D1-1-8_mid--AUDIO-from_mp3_08_R1_2015_wav--3
Processing file  257 of  279
pcts: 8.778195488721805 19.4117

pcts: 7.857142857142857 20.392156862745097 28.37837837837838 48.148148148148145
multipcts: [48.148148148148145, 64.40677966101694, 55.26315789473685, 52.305475504322764]


In [ ]:
#passos: juntar pcts_over com pcts_over_smart
#substituir os arquivos presentes em pcts_over na tabela maior pcts 

In [142]:
pcts_1 = pd.read_csv('./results/pcts_new_over_smart.csv', sep=';')
pcts_2 = pd.read_csv('./results/pcts_new_over.csv', sep=';')

In [143]:
pcts_2.drop(['pct_multilevel_2','pct_multilevel_3'],axis=1,inplace=True)
pcts_new = pcts_2.append(pcts_1, ignore_index=True)

In [144]:
pcts_new.head()

,file name,pct_200,pct_500,pct_800,pct_1600,pct_multilevel_4
0,/Users/nicolas/Documents/ime/compmus/mestrado/...,3.552632,8.235294,14.864865,28.703704,"[28.703703703703702, 62.857142857142854, 53.19..."
1,/Users/nicolas/Documents/ime/compmus/mestrado/...,4.304511,9.019608,12.612613,19.444444,"[19.444444444444446, 73.80952380952381, 58.571..."
2,/Users/nicolas/Documents/ime/compmus/mestrado/...,7.706767,20.588235,27.927928,45.370370,"[45.370370370370374, 66.12903225806451, 63.803..."
3,/Users/nicolas/Documents/ime/compmus/mestrado/...,8.740602,20.588235,27.252252,43.518519,"[43.51851851851852, 63.46153846153846, 61.3333..."
4,/Users/nicolas/Documents/ime/compmus/mestrado/...,3.815789,6.862745,11.486486,25.000000,"[25.0, 46.2962962962963, 50.847457627118644, 6..."


In [145]:
pcts_old = pd.read_csv('./results/pcts.csv', sep=';')
pcts_old.drop(['pct_multilevel_2','pct_multilevel_3'],axis=1,inplace=True)

In [146]:
pcts_old.head()

,file name,pct_200,pct_500,pct_800,pct_1600,pct_multilevel_4
0,/Users/nicolas/Documents/ime/compmus/mestrado/...,14.830827,23.235294,29.054054,42.592593,"[42.592592592592595, 82.5, 85.80246913580247, ..."
1,/Users/nicolas/Documents/ime/compmus/mestrado/...,14.135338,16.470588,25.675676,42.592593,"[42.592592592592595, 64.13043478260869, 73.856..."
2,/Users/nicolas/Documents/ime/compmus/mestrado/...,4.774436,9.411765,10.135135,18.518519,"[18.51851851851852, 72.5, 56.86274509803921, 6..."
3,/Users/nicolas/Documents/ime/compmus/mestrado/...,10.902256,17.254902,21.396396,35.185185,"[35.18518518518518, 72.09302325581395, 71.0743..."
4,/Users/nicolas/Documents/ime/compmus/mestrado/...,28.928571,34.313725,32.657658,33.333333,"[33.33333333333333, 95.83333333333334, 100.0, ..."


In [147]:
idx_over = pcts_old[pcts_old['pct_1600'] > 45.0].index

In [149]:
print(len(pcts_new), len(idx_over))

279 279


In [150]:
j = 0
for idx in idx_over:
    if pcts_old.loc[idx]['file name'] == pcts_new.iloc[j]['file name']:
        pcts_old.loc[idx] = pcts_new.iloc[j]
    else:
        print('deu errado')
    j += 1

In [156]:
pcts_all_final = pcts_old

Agora: criar colunas a mais pro pcts_all_final, salvar em csv p ser usado em experimento

In [159]:
pcts_all_final['pct_multilevel_4'].iloc[0]

'[42.592592592592595, 82.5, 85.80246913580247, 83.40380549682875]'

In [202]:
def to_list(list_of_strings):
    return list(map(float, list_of_strings.strip('][').split(', ')))

def treat_pcts(pcts):
    for col in ['pct_multilevel_4', 'pct_multilevel_3', 'pct_multilevel_2']:
        pcts[col] = pcts[col].apply(to_list)
    return pcts

def get_pcts(pcts, file):
    pcts_file = (pcts[pcts['file name'] == file]).iloc[0]
    return(pcts_file['pct_200'], pcts_file['pct_500'], pcts_file['pct_800'], pcts_file['pct_1600'],
            pcts_file['pct_multilevel_2'], pcts_file['pct_multilevel_3'], pcts_file['pct_multilevel_4'])

In [162]:
pcts_all_final = treat_pcts(pcts_all_final)

In [163]:
pcts_all_final.head()

,file name,pct_200,pct_500,pct_800,pct_1600,pct_multilevel_4
0,/Users/nicolas/Documents/ime/compmus/mestrado/...,14.830827,23.235294,29.054054,42.592593,"[42.592592592592595, 82.5, 85.80246913580247, ..."
1,/Users/nicolas/Documents/ime/compmus/mestrado/...,14.135338,16.470588,25.675676,42.592593,"[42.592592592592595, 64.13043478260869, 73.856..."
2,/Users/nicolas/Documents/ime/compmus/mestrado/...,4.774436,9.411765,10.135135,18.518519,"[18.51851851851852, 72.5, 56.86274509803921, 6..."
3,/Users/nicolas/Documents/ime/compmus/mestrado/...,10.902256,17.254902,21.396396,35.185185,"[35.18518518518518, 72.09302325581395, 71.0743..."
4,/Users/nicolas/Documents/ime/compmus/mestrado/...,28.928571,34.313725,32.657658,33.333333,"[33.33333333333333, 95.83333333333334, 100.0, ..."


In [165]:
pcts_all_final.iloc[0]['pct_multilevel_4'][0]

42.592592592592595

In [168]:
pcts_all_final['pct_multilevel_3'] = pcts_all_final['pct_multilevel_4'].apply(lambda x: x[:3])
pcts_all_final['pct_multilevel_2'] = pcts_all_final['pct_multilevel_4'].apply(lambda x: x[:2])

In [178]:
pcts_all_final.to_csv('./results/pcts_all_final.csv')